In [35]:
import pyterrier as pt

if not pt.started():

    pt.init(tqdm="notebook")

In [36]:
dataset = pt.get_dataset("irds:beir/fiqa")

In [37]:
index_path = "./sparse_index_fiqa"
# Load index to memory
index = pt.IndexFactory.of(index_path, memory=True)

In [38]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [39]:
testset = pt.get_dataset("irds:beir/fiqa/test")

In [40]:
bm25 = bm25 % 100

In [41]:
first_stage_res = bm25.transform( testset.get_topics())

In [42]:
from fast_forward.encoder import TCTColBERTQueryEncoder, TCTColBERTDocumentEncoder
import torch

q_encoder = TCTColBERTQueryEncoder("castorini/tct_colbert-msmarco")
d_encoder = TCTColBERTDocumentEncoder(
    "castorini/tct_colbert-msmarco",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

In [43]:
from fast_forward import OnDiskIndex, Mode
from pathlib import Path

ff_index = OnDiskIndex.load(
    Path("./dense_index_fiqa_GTE-base/ffindex_fiqa_gte-base-en-v1.5.h5"), query_encoder=q_encoder, mode=Mode.MAXP
).to_memory()

100%|██████████| 57638/57638 [00:00<00:00, 144974.64it/s]


In [44]:
from fast_forward.util.pyterrier import FFScore
from fast_forward.util.pyterrier import FFInterpolate

ff_score = FFScore(ff_index)
ff_int = FFInterpolate(alpha=0.05) #added the optimal version from the beginning in order to avoid running GridsSearch again

In [45]:
reranking_stage = ff_score >> ff_int

In [46]:
def time_reranking():
    reranking_stage.transform(first_stage_res)

In [47]:
import timeit
execution_time = timeit.timeit(time_reranking, number=3)
print(execution_time)

77.57979716499904
